In [1]:
import pandas as pd
import shapely
import geopandas as gp
import json

In [2]:
layers = {}
layers["Borough"] = gp.GeoDataFrame.from_file("Borough Boundaries.geojson")
layers["Community"] = gp.GeoDataFrame.from_file("Community Districts.geojson")
layers["NTA"] = gp.GeoDataFrame.from_file("Neighborhood Tabulation Areas.geojson")
layers["Tract"] = gp.GeoDataFrame.from_file("2010 Census Tracts.geojson")
list = layers.keys()

In [34]:
# In order to make the site load quicker, it is
# necessary to be able to push only those features
# which are within the display area and to have
# lower resolution layers for when the display is
# a larger area.

# If geopandas was on PythonAnywhere, it would not
# be necessary to create these in advance. However,
# since it is not included by default, and I do not
# have disk space there, this is a needed workaround.
proj = {}
for cov in list:
    proj[cov] = layers[cov].to_crs(epsg=2263)
    envel = proj[cov].geometry.envelope
    enveljs = json.loads(envel.to_json())
    # An envelope field cannot be converted directly to
    # json, so create fields for the corners of the
    # envelope and add those
    # This will allow measuring the extent of what is displayed
    # and testing if a feature has any overlap with it.
    bounds = [[enveljs['features'][i]['bbox'][j] for i in range(len(enveljs['features']))] for j in range(4)]
    proj[cov]['minx'] = bounds[0]
    proj[cov]['miny'] = bounds[1]
    proj[cov]['maxx'] = bounds[2]
    proj[cov]['maxy'] = bounds[3]
    for res in [10, 20, 50, 100, 200, 400, 800]:
        scov = "{:s}{:d}".format(cov, res)
        proj[scov] = proj[cov]
        proj[scov].geometry = proj[cov].geometry.simplify(res)
        js = proj[scov].to_json()
        with open('resolutions/{}.geojson'.format(scov), 'w') as out:
            json.dump(js, out)